In [ ]:
ll=[-40,40]
a=array([[0,len(lon)],[0,len(lat)]])
####ERROR HERE I NEED TO INTERP THE ARAKAWA B GRID
eddys=scan_eddym(eta[:,:,:],lon,lat,ll,0,a,okparm,diagnostics=False)
print shape(eddys['Position'])

In [ ]:
print eddys['EddyN'][:]
print shape(eddys['EddyN'][:])
for ii in range(0,5):
    if ii==0:
        eddytd=dict_eddyt(ii,eddys)
    else:
        eddytd=dict_eddyt(ii,eddys,eddytd)

In [ ]:
for key, value in eddytd.items() :
    print (key,value)

In [ ]:
print shape(eddys['Position'])
print shape(eddys['Ellipse'])
eddypos=eddys['Position']
print eddypos[0]
figure(figsize=(15,5))
pcolormesh(lon,lat,eta[0,:,:])
for ii in range(0,shape(eddys['Position'])[0]):
    plot(eddypos[ii][0],eddypos[ii][1],'or')

In [ ]:
dda=0
tt=0
for ii in range(0,shape(dat0)[0]):
    datad=dat0[ii]
    for jj in range(0,shape(datad)[0]):
        dd=datad[jj]
        if (len(dd[:,0]) | len(dd[:,1])) <= 8:
            print 'Singular index I cant get an ellipse with this data'
            xx=nan
            yy=nan
            center=[nan,nan]
            check=0
        else:
            a = fitEllipse(dd[:,0],dd[:,1])
            center = ellipse_center(a)
            phi = ellipse_angle_of_rotation(a)
            axes = ellipse_axis_length(a)
            
            #print("center = ",  center)
            #print("angle of rotation = ",  phi)
            #print("axes = ", axes)
            
            R = np.arange(0,2*pi, 0.1)
            a,b = axes
            
            eccen=eccentricity(a,b)
            #print 'ecce'+str(eccen)
            
            xx = center[0] + a*np.cos(R)*np.cos(phi) - b*np.sin(R)*np.sin(phi)
            yy = center[1] + a*np.cos(R)*np.sin(phi) + b*np.sin(R)*np.cos(phi)
            
            contarea=PolyArea(dd[:,0],dd[:,1])
            elipsarea=PolyArea(xx,yy)
            #print contarea, elipsarea
            countzeros=0
            for ii in range(0,len(dd[:,0])):
                idxcheck,idycheck=find2d(lon,lat,dd[ii,0],dd[ii,1])
                idxelipcheck,idyelipcheck=find2d(lon,lat,center[0],center[1])
                #print lon[idxelipcheck],lat[idyelipcheck]
                #print ncdatanm[tt,idxelipcheck,idyelipcheck]
                
                
                if ncdatanm[tt,idxcheck,idycheck]==nan:
                    countzeros=countzeros+1
            if countzeros>=len(dd[:,0])/2:
                print 'Thisone is land'
                check=0
            else:
                
                if contarea>elipsarea:
                    if contarea/2>elipsarea:
                        print 'Removing contour, thisone is really underestimate'
                        check=0
                    else:
                        print 'Saving contour in path:'
                        chech=1
                elif contarea<elipsarea:
                    if contarea<elipsarea/2:
                        print 'Removing contour, thisone is really overestimate'
                        check=0
                    elif eccen<0.99:
                        print 'Saving contour in path:'
                        check=1
                    else:
                        print 'Removing contour, thisone is really underestimate'
                        check=0
                    #Eccentricity
            if check==1:# or check==0:
                f, (ax1, ax2) = plt.subplots(1, 2,figsize=(13, 6))
                ax1.contourf(lon[0:400],lat[100:600],ncdata[0,100:600,0:400])
                ax1.plot(dd[:,0],dd[:,1],'*r')
                ax1.plot(xx,yy,'-b')
                ax1.plot(center[0],center[1],'ob')
                cc=ax2.contourf(lon[0:400],lat[100:600],ncdata[0,100:600,0:400])
                f.subplots_adjust(right=0.8)
                cbar_ax = f.add_axes([0.85, 0.15, 0.05, 0.7])
                f.colorbar(cc, cax=cbar_ax)
                ax2.plot(dd[:,0],dd[:,1],'*r')
                ax2.plot(xx,yy,'-b')
                ax2.plot(center[0],center[1],'ob')
                x, y = np.meshgrid(lon[0:400],lat[100:600])
                c = np.ones_like(x)
                ax2.plot(lon[idxelipcheck],lat[idyelipcheck],'om')
                ax2.pcolor(x, y, c, facecolor='none', edgecolor='k')
                ax2.set_ylim([dd[:,1].min(),dd[:,1].max()])
                ax2.set_xlim([dd[:,0].min(),dd[:,0].max()])
                plt.show()
                plt.close()
        dda=dda+1
print dda

In [ ]:
#print eddytd['eddyn_1']['contour']
#mask function V1

ixnan1=[]
ixnan2=[]
threshold=1
resolution=0.1
for ct in range(0,len(eddytd['eddyn_1']['time'])):
    rct=eddytd['eddyn_1']['time'][ct][0]
    data=eta[rct,:,:]
    juesteddy=zeros(shape(data))
    lonmi=eddytd['eddyn_1']['contour'][ct][0].min()
    lonma=eddytd['eddyn_1']['contour'][ct][0].max()
    latmi=eddytd['eddyn_1']['contour'][ct][1].min()
    latma=eddytd['eddyn_1']['contour'][ct][1].max()
    #print latma, lonma
    mimcx,mimcy=find2d(lon,lat,lonmi,latmi)
    mamcx,mamcy=find2d(lon,lat,lonma,latma)
    
    loncm=lon[mimcx-threshold:mamcx+1+threshold]
    latcm=lat[mimcy-threshold:mamcy+1+threshold]
    
    datacm=data[mimcy-threshold:mamcy+1+threshold,mimcx-threshold:mamcx+1+threshold]
    minlonst=find(eddytd['eddyn_1']['contour'][ct][0],lonmi)
    latmean = mean(eddytd['eddyn_1']['contour'][ct][1][minlonst:len(eddytd['eddyn_1']['contour'][ct][0])/2])
    #print midlatidy,len(eddytd['eddyn_1']['contour'][ct][0])
    figure()
    #plot(loncm[minlonst],latcm[minlonst],'oc')
    lastloncheck=[eddytd['eddyn_1']['contour'][ct][0][minlonst]-resolution]
    for cy in range(0,(len(eddytd['eddyn_1']['contour'][ct][0])/2)+1):
        if minlonst+cy == (len(eddytd['eddyn_1']['contour'][ct][0])):
            minlonst = -cy
        # Look for the nearest point uper function and lower function
        
        loncomp=find(loncm,eddytd['eddyn_1']['contour'][ct][0][minlonst+cy])
        latcomp=find(latcm,eddytd['eddyn_1']['contour'][ct][1][minlonst+cy])
        loncomn=find(loncm,eddytd['eddyn_1']['contour'][ct][0][minlonst-cy])
        latcomn=find(latcm,eddytd['eddyn_1']['contour'][ct][1][minlonst-cy])
        #if mean(eddytd['eddyn_1']['contour'][ct][1][minlonst:(len(eddytd['eddyn_1']['contour'][ct][0])/2)+1]) >= latmean:
        
        for ry in range(0,latcomp-threshold):
        #for ry in range(latcomp+threshold,len(latcm)):
                #datacm[ry,loncomp]=nan
            #m= (latcm[cy]-latcm[cy-1])
            print lastloncheck[:]
            print abs(lastloncheck[-1]-eddytd['eddyn_1']['contour'][ct][0][minlonst+cy])
            if abs(lastloncheck[-1]-eddytd['eddyn_1']['contour'][ct][0][minlonst+cy])>resolution/2 :
                plot(loncm[loncomp],latcm[ry],'*w')
                lastloncheck.append(eddytd['eddyn_1']['contour'][ct][0][minlonst+cy])
        #elif latcm[latcomp+1] > latmean:
        #    for ry in range(latcomp+threshold,len(latcm)):
        #        #datacm[ry,loncomp]=nan
        #        plot(loncm[loncomp],latcm[ry],'*w')
        
        
        #else:
        #    for ry in range(latcomp+threshold,len(latcm)):
                #datacm[ry,loncomp]=nan
        #        plot(loncm[loncomp],latcm[ry],'*w')
        #elif latcomp < latmean:
        #    for ry in range(0,latcomn-threshold):
        #        datacm[ry,loncomp]=nan
        #        plot(loncm[loncomp],latcm[ry],'*m')
        
        #print eddytd['eddyn_1']['contour'][ct][1][midlatidy+cy]
        #threshold
        #if lat[latcomp] > eddytd['eddyn_1']['contour'][ct][1][midlatidy+cy]\
        #    and mean(eddytd['eddyn_1']['contour'][ct][1][midlatidy:midlatidy+\
        #                len(eddytd['eddyn_1']['contour'][ct][0])/2])- latmean > 0:
        #if 
        
        pcolormesh(lon[mimcx-threshold:mamcx+1+threshold],lat[mimcy-threshold:mamcy+1+threshold],datacm)
        contourf(lon[mimcx-threshold:mamcx+1+threshold],lat[mimcy-threshold:mamcy+1+threshold],datacm,alpha=0.5)
        #plot(lonmesh,latmesh,'-k')
        plot(lon[mimcx-1],lat[mimcy-1],'ok')
        plot(lon[mamcx+1],lat[mamcy+1],'ok')
        plot(eddytd['eddyn_1']['contour'][ct][0],eddytd['eddyn_1']['contour'][ct][1],'-m')
        plot(eddytd['eddyn_1']['contour'][ct][0][minlonst+cy],eddytd['eddyn_1']['contour'][ct][1][minlonst+cy],'or')
        plot(eddytd['eddyn_1']['contour'][ct][0][minlonst-cy],eddytd['eddyn_1']['contour'][ct][1][minlonst-cy],'ob')
        figure()
        datacm[datacm<=-79.9]=nan
        pcolormesh(lon[mimcx-threshold:mamcx+1+threshold],lat[mimcy-threshold:mamcy+1+threshold],datacm)
        plt.show()
        ##CHECK how to locate the points in their respective location.
        